In [ ]:
%cd drive/MyDrive/Colab Notebooks/big data sample

/content/drive/MyDrive/Colab Notebooks/big data sample


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 取得全台有7-11的各縣市名稱


In [ ]:
url = 'https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
area_total = soup.find('select', id='Class1')
area_total

<select class="formstyle width170" id="Class1" name="FirstClass" onchange="getRetailArea('retail_inquiry_control.aspx', 'thisform', '1', '2', 2);" size="1">
<option value="1*台北市">台北市</option>
<option value="2*新北市">新北市</option>
<option value="3*基隆市">基隆市</option>
<option value="4*宜蘭縣">宜蘭縣</option>
<option value="5*桃園市">桃園市</option>
<option value="6*新竹市">新竹市</option>
<option value="7*新竹縣">新竹縣</option>
<option value="8*苗栗縣">苗栗縣</option>
<option value="9*台中市">台中市</option>
<option value="10*彰化縣">彰化縣</option>
<option value="11*南投縣">南投縣</option>
<option value="12*雲林縣">雲林縣</option>
<option value="13*嘉義市">嘉義市</option>
<option value="14*嘉義縣">嘉義縣</option>
<option value="15*台南市">台南市</option>
<option value="16*高雄市">高雄市</option>
<option value="17*屏東縣">屏東縣</option>
<option value="18*花蓮縣">花蓮縣</option>
<option value="19*台東縣">台東縣</option>
<option value="20*澎湖縣">澎湖縣</option>
<option value="21*金門縣">金門縣</option>
<option value="22*連江縣">連江縣</option>
<option value="23*南海諸島">南海諸島</option>
</select>

In [ ]:
areas = area_total.find_all('option')
areas[0]

<option value="1*台北市">台北市</option>

In [ ]:
area_name = []
for i in range(len(areas)):
  area_name.append(areas[i].text)

print(len(area_name))
print(area_name[:5])  

23
['台北市', '新北市', '基隆市', '宜蘭縣', '桃園市']


# 以連江縣為例找到儲存7-11門市資料的方法

In [ ]:
url = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
payload = {'strTargetField': 'COUNTY', 'strKeyWords': '南海諸島'}

html = requests.post(url, data=payload)

**利用pd內建的read_html讀取網頁上表格資料**

In [ ]:
#Pandas套件提供了read_html()方法(Method)，只要傳入網址，就能夠讀取網頁中的<table>表格標籤
###header=0:索引的第0列為表頭
data = pd.read_html(html.text, header=0)[0]
data

,店號,店名,地址
0,無符合條件的門市資料，請重新定義條件,無符合條件的門市資料，請重新定義條件,無符合條件的門市資料，請重新定義條件


**利用tr和td標籤慢慢把每間7-11門市的資料取出來**

In [ ]:
soup = BeautifulSoup(html.text, 'html.parser')
datas = soup.find_all('tr')

#print(datas[0:3])
print(datas[1].text)

無符合條件的門市資料，請重新定義條件


In [ ]:
df = []
try:
  for j in range(1,len(datas)):
    name = datas[j].find_all('td')
    info = []
    for i in name:
      info.append(i.text.strip())
    df1 = pd.DataFrame([{'店號':info[0], '店名':info[1], '地址':info[2]}])

    df.append(df1)
except:
  if datas[1].text == '無符合條件的門市資料，請重新定義條件':
    info = '無符合條件的門市資料，請重新定義條件'
    df1 = pd.DataFrame([{'店號':info, '店名':info, '地址':info}])
    df.append(df1)
  else:
    print('資料有錯誤!!!')  

if df != []:
  df_all = pd.concat(df, ignore_index=True)
  print(df_all)
else:
  print('無法建立7-11門市資訊!!!')  

,店號,店名,地址
0,無符合條件的門市資料，請重新定義條件,無符合條件的門市資料，請重新定義條件,無符合條件的門市資料，請重新定義條件


# 把每個縣市的7-11門市資訊儲存成Excel檔

In [ ]:
###ExcelWriter可以在一個excel檔案中建立不同的sheet並寫入資料
with pd.ExcelWriter('711/7-11.xlsx') as writer:
  for name in area_name:
    url = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
    payload = {'strTargetField': 'COUNTY', 'strKeyWords': name}
    html = requests.post(url, data=payload)
    
    soup = BeautifulSoup(html.text, 'html.parser')
    datas = soup.find_all('tr')
    
    print('{}資料下載中'.format(name))  
    df = []
    try:
      for j in range(1,len(datas)):
        items = datas[j].find_all('td')
        info = []
        for i in items:
          info.append(i.text.strip())
        df1 = pd.DataFrame([{'店號':info[0], '店名':info[1], '地址':info[2]}])

        df.append(df1)
    except:
      if datas[1].text == '無符合條件的門市資料，請重新定義條件':
        info = '無符合條件的門市資料，請重新定義條件'
        df1 = pd.DataFrame([{'店號':info, '店名':info, '地址':info}])
        df.append(df1)
      else:
        print('{}資料有錯誤!!!'.format(name))  

    if df != []:
      df_all = pd.concat(df, ignore_index=True)
    else:
      print('{}無法建立7-11門市資訊!!!'.format(name)) 
    
    ###index=False:不顯示索引列
    df_all.to_excel(writer, sheet_name=name, index=False)

print('全台資料下載完成!!')        

台北市資料下載中
新北市資料下載中
基隆市資料下載中
宜蘭縣資料下載中
桃園市資料下載中
新竹市資料下載中
新竹縣資料下載中
苗栗縣資料下載中
台中市資料下載中
彰化縣資料下載中
南投縣資料下載中
雲林縣資料下載中
嘉義市資料下載中
嘉義縣資料下載中
台南市資料下載中
高雄市資料下載中
屏東縣資料下載中
花蓮縣資料下載中
台東縣資料下載中
澎湖縣資料下載中
金門縣資料下載中
連江縣資料下載中
南海諸島資料下載中
全台資料下載完成!!


# 觀察每個縣市7-11門市數量


In [ ]:
df = pd.read_excel('711/7-11.xlsx', sheet_name=None)
num = 0
rank = {}
for name, info in df.items():
  num += 1
  ###orient='records':以list的形式把每個index的內容存起來
  info1 = info.to_dict(orient='records')
  if len(info1) == 1:
    if '無符合條件的門市資料，請重新定義條件' in info1[0].values():
         print('{}. {}的7-11門市有{}間'.format(num,name,len(info1)-1))
         rank[name] = len(info1)-1
         continue
  print('{}. {}的7-11門市有{}間'.format(num,name,len(info1)))
  rank[name] = len(info1)

print('..................................')
print('分析如下:')
maxi = max(rank, key=rank.get)
mini = min(rank, key=lambda x:rank[x])
print('{}有最多的7-11門市'.format(maxi))
print('{}有最少的7-11門市'.format(mini))

1. 台北市的7-11門市有882間
2. 新北市的7-11門市有1119間
3. 基隆市的7-11門市有86間
4. 宜蘭縣的7-11門市有117間
5. 桃園市的7-11門市有749間
6. 新竹市的7-11門市有165間
7. 新竹縣的7-11門市有200間
8. 苗栗縣的7-11門市有149間
9. 台中市的7-11門市有838間
10. 彰化縣的7-11門市有242間
11. 南投縣的7-11門市有101間
12. 雲林縣的7-11門市有130間
13. 嘉義市的7-11門市有67間
14. 嘉義縣的7-11門市有107間
15. 台南市的7-11門市有548間
16. 高雄市的7-11門市有671間
17. 屏東縣的7-11門市有179間
18. 花蓮縣的7-11門市有106間
19. 台東縣的7-11門市有63間
20. 澎湖縣的7-11門市有35間
21. 金門縣的7-11門市有24間
22. 連江縣的7-11門市有12間
23. 南海諸島的7-11門市有0間
..................................
分析如下:
新北市有最多的7-11門市
南海諸島有最少的7-11門市
